# JS customization

Table Of Contents :

+ [Custom Buttons](#Building-custom-buttons)
+ [Custom JS](#Completely-custom-js)
+ [Linking Widgets](#Linking-widgets-together)
    - [Watching updates from outside the grid](#Updating-information-in-the-grid-via-outside-widgets)
    - [Watching the data of the grid for interactive plotting](#Using-the-interactivity-of-the-grid-outputs-for-interactive-plotings)

In [1]:
import os
import json
import numpy as np
import pandas as pd
import urllib.request as ur
import ipywidgets as widgets
from ipyaggrid import Grid


In [2]:
url = 'https://raw.githubusercontent.com/widgetti/ipyaggrid/master/data/OlympicWinners.json'
with ur.urlopen(url) as res:
    data = json.loads(res.read().decode('utf-8'))


## Building custom buttons

Buttons are an easy way to interact with the grid. They can perform many actions on the gridOptions, and many examples of them are already given in [ag-Grid documentation](https://www.ag-grid.com/documentation-main/documentation.php), so that you can test anyfeature you want using their Plunckers (in browsers test environments).

This custom button is built using two parameters : the name of the button and the action of the button. The first one will be used to be displayed on the button, and the second one is the body of the function called on click.
The code **can use the `gridOptions`** as if they were defined before, and thus access to the `api` and the `columnApi` of ag-Grid.

With this custom button, two cells are highlighted if their difference is higher than 50.

In [3]:
columnDefs = [
    {'headerName': "Country", 'field': "country", 'width': 120, 'rowGroup': 'true'},
    {'headerName': "Year", 'field': "year", 'width': 90, 'pivot': 'true', 'enablePivot':True},
    {'headerName': "Sport", 'field': "sport", 'width': 110, 'rowGroup': 'true'},
    {'headerName': "Athlete", 'field': "athlete"},
    {'headerName': "Gold", 'field': "gold", 'width': 100, 'aggFunc': 'sum'},
];

for c in columnDefs:
    c["sortable"] = True
    c["filter"]=True
    c["resizable"]=True


gridOptions = {
    'pivotMode': 'true',
    'columnDefs': columnDefs,
    'animateRows':'true',
};

buttons=[{'name':'Highlight', 'action':"""
        var count = gridOptions.api.getDisplayedRowCount();
        for (var i = 0; i<count; i++) {
          var rowNode = gridOptions.api.getDisplayedRowAtIndex(i);
          if(rowNode.aggData != null && Object.keys(rowNode.aggData).length > 0){
        var keys = Object.keys(rowNode.aggData);
        var gold = [];
        for (var k = 0; k<keys.length; k++){
          var j = 2*k + 1;
          var prop = "pivot_" + j;
          if(rowNode.aggData[prop] == null){
            rowNode.aggData[prop] = 0;
          }
          gold[k] = rowNode.aggData[prop];
        }
        for(var j=0;j<gold.length - 1;j++){
          if(Math.abs(gold[j] - gold[j+1]) >= 50){
            var column1 = "pivot_" + (2*j+1);
            var column2 = "pivot_" + (2*(j+1)+1);
            gridOptions.api.flashCells({rowNodes: [rowNode], columns: [column1, column2] });
          }
        }}}"""}]


grid1 = Grid(quick_filter=True,
             theme='ag-theme-balham',
             compress_data=True,
             menu={'buttons':buttons},
             grid_options=gridOptions,
             grid_data=data,
             columns_fit="auto")
grid1

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

## Completely custom js - Pay attention: <span style="color:red">A lot of tips here</span>
### Will not work in JLab

The example below shows how to:
+ Insert a column computed from the input data
+ Set [cell style](https://www.ag-grid.com/javascript-grid-cell-styling/):
    + **Age** 30 and above is ahow in red
+ Set [cell renderers](https://www.ag-grid.com/javascript-grid-cell-rendering/):
    + **Sport** is displayed with a button: Open console to see the data collected upon click
    + **Score** is displayed with bar in the background 
+ Use the [**ag-Grid** context object](https://www.ag-grid.com/javascript-grid-context/) to carry precalculated and grid (as opposed to row) level info
+ Use the various JS injection points:
    + `js_helpers_custom`: Define a new helper function
    + `js_pre_grid`: Enrich ColumnDefs before grid instantiation
    + `js_post_grid`: Determine values (`max_score`) and functions (`scale`) from all data
    
    
+ A **score** is calculated from several columns:
    + $\sqrt{\displaystyle(\text{3 gold}^2+\text{2 silver}^2+\text{1 bronze}^2)}$. 
    + **Note**: It does not represent any real world value, just a pretext for the background bars. 


In [4]:
age_cell_style = """
function(params) { if (params.value >= 30) { return {color: 'red'}; } }
"""

sport_cell_renderer = """
function (params){
    let v = params.value;

    function clicked(){
        //window.params = params;
        let scale = params.api.context.my_data.my_scale;
        
        console.log('row data:');
        console.log(params.data);
        console.log('row index:');
        console.log(params.rowIndex);
        console.log('score:');
        console.log(params.data.score);
        console.log('max_score (from context):');
        console.log(params.api.context.my_data.max_score);
    }

    let b = document.createElement('button');
    b.innerHTML = 'Click';
    b.style = "background-color:bisque; margin:1px 10px 1px 2px;";
    b.title = "Open console after click";
    b.addEventListener("click", function (){clicked()}, false);
    
    let d = document.createElement('div');
    d.style = 'display: flex';
    let d2 = document.createElement('div');
    d2.innerHTML = v;
    d.appendChild(b);
    d.appendChild(d2);

    return d;
}
"""

columnDefs = [
    {'headerName': "Athlete", 'field': "athlete", 'width': 150},
    {'headerName': "Age", 'field': "age", 'width': 90, 'cellStyle': age_cell_style},
#     {'headerName': "Country", 'field': "country", 'width': 120},
    {'headerName': "Year", 'field': "year", 'width': 90},
#     {'headerName': "Date", 'field': "date", 'width': 145},
    {'headerName': "Sport", 'field': "sport", 'width': 180, 'cellRenderer': sport_cell_renderer},
    {'headerName': "Gold", 'field': "gold", 'width': 70},
    {'headerName': "Silver", 'field': "silver", 'width': 75},
    {'headerName': "Bronze", 'field': "bronze", 'width': 85}
]

js_helpers_custom="""
helpersCustom = {
    formatFloat3: d3.format(',.3f')
}
"""

js_pre_grid = ["""
window.go = gridOptions;
function scoreCellRenderer(params){
    let v = params.value;
    let scale = params.api.context.my_data.my_scale;
    let f = scale(v);
    let css = `background: linear-gradient(to right, #bcbddc ${f}%, transparent ${f}%); ; flex-grow:2`;
    let html = `<div style="${css}">${helpers.formatFloat3(v)}</div>` 
    return html;
}
gridOptions.columnDefs.push({'headerName':'Score', field:'score', cellRenderer: scoreCellRenderer});
gridData.forEach(dat => {
    dat.score = Math.sqrt(Math.pow(3*dat.gold,2)+
                          Math.pow(2*dat.silver,2)+
                          Math.pow(1*dat.bronze,2))
})
"""]

js_post_grid = ["""
let arr_score = [];
gridOptions.api.forEachNode(node => {arr_score.push(node.data.score)});
let max_score = Math.max(...arr_score);
let scale = d3.scaleLinear().domain([0, max_score]).range([0, 100])
gridOptions.api.context.my_data = {my_scale: scale, max_score: max_score};
"""]

grid_options = {
    'columnDefs': columnDefs,
    'defaultColDef': {'filter': 'true', 'sortable': 'true'}
}

grid2 = Grid(grid_data = data,
            grid_options=grid_options,
            js_helpers_custom=js_helpers_custom,
            js_pre_grid=js_pre_grid,
            js_post_grid=js_post_grid,
            export_mode='disabled',
            theme='ag-theme-balham')
grid2

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='350px', js_helpers_custom=…

## Linking widgets together

### Updating information in the grid via outside widgets

This example explains how to use `user_params` with other widget changes to update the effect of some action on the grid. 
Here we have a JS function highlight that flashes some cells of the grid when clicking one button on certain conditions depending on a parameter. We would like to choose this parameter via a slider.

We could of course build this slider directly inside the JS of the grid using the `to_eval` input parameter. However building such JS components can sometimes be very tedious. We would prefer to ***use an ipywidget*** to build it, like the IntSlider.



In [5]:
columnDefs = [
    {'headerName': "Country", 'field': "country", 'width': 120, 'rowGroup': 'true'},
    {'headerName': "Year", 'field': "year", 'width': 90, 'pivot': 'true'},
    {'headerName': "Sport", 'field': "sport", 'width': 110, 'rowGroup': 'true'},
    {'headerName': "Athlete", 'field': "athlete"},
    {'headerName': "Gold", 'field': "gold", 'width': 100, 'aggFunc': 'sum'},
];

gridOptions = {
    'pivotMode': 'true',
    'defaultColDef': {'sortable': 'true', 'filter': 'true', 'resizable': 'true'},
    'columnDefs': columnDefs,
    'animateRows':'true',
};

buttons=[{'name':'Highlight', 'action':"""
        var count = view.gridOptions.api.getDisplayedRowCount();
        for (var i = 0; i<count; i++) {
          var rowNode = view.gridOptions.api.getDisplayedRowAtIndex(i);
          if(rowNode.aggData != null && Object.keys(rowNode.aggData).length > 0){
        var keys = Object.keys(rowNode.aggData);
        var gold = [];
        for (var k = 0; k<keys.length; k++){
          var j = 2*k + 1;
          var prop = "pivot_" + j;
          if(rowNode.aggData[prop] == null){
            rowNode.aggData[prop] = 0;
          }
          gold[k] = rowNode.aggData[prop];
        }
        for(var j=0;j<gold.length - 1;j++){
          // test positive if absolute value of difference of sum of gold medals
          // from one olympiad to the next is greater than or equal to sider_value
          if(Math.abs(gold[j] - gold[j+1]) >= view.model.get('user_params').slider_value){
            var column1 = "pivot_" + (2*j+1);
            var column2 = "pivot_" + (2*(j+1)+1);
            view.gridOptions.api.flashCells({rowNodes: [rowNode], columns: [column1, column2] });
          }
        }}}"""}]


pivot = Grid(quick_filter=True,
             theme='ag-theme-balham',
             compress_data=True,
             menu = {'buttons':buttons},
             grid_options=gridOptions,
             grid_data=data,
             columns_fit="auto",
             user_params={'slider_value':50})
pivot

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

With this function in `buttons`, two adjacent cells are highlighted if their difference in absolute value is higher than `user_parameter.slider_value`.

In [6]:
# Setting a simple slider to coordinate its value with the highlight button
slider = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Highlight Value:',
    style={'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def on_slider_change(change):
    pivot.user_params = {'slider_value': change.new}

slider.observe(on_slider_change, names='value')

The custom JS function we built uses `view.model.get('user_params')['slider_value']` instead of the regular hardcoded value. As the value of the slider changes, so does the `user_params`, using a simple observe.

In [7]:
slider

IntSlider(value=50, continuous_update=False, description='Highlight Value:', style=SliderStyle(description_wid…

In [8]:
pivot

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', js_helpers='{"dateFormatt…

### Using the interactivity of the grid outputs for interactive plotings

Let's try to do some kind of opposite situation. We are going to link a dataset displayed by a [bqplot](https://github.com/bloomberg/bqplot) widget, with the output of a grid.

In [9]:
from bqplot import pyplot as plt
from bqplot import (
    Axis, ColorAxis, LinearScale, DateScale, DateColorScale, OrdinalScale,
    OrdinalColorScale, ColorScale, Scatter, Lines, Figure, Tooltip
)

In [10]:
# Copy pasted from bqplot scatter documentation
price_data = pd.DataFrame(np.cumsum(np.random.randn(150, 2).dot([[1.0, -0.8], [-0.8, 1.0]]), axis=0) + 100,
                          columns=['Security 1', 'Security 2'], index=pd.date_range(start='01-01-2007', periods=150))
size = 100
np.random.seed(0)
x_data = range(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)
ord_keys = np.array(['A', 'B', 'C', 'D', 'E', 'F'])
ordinal_data = np.random.randint(5, size=size)
symbols = ['Security 1', 'Security 2']

dates_all = price_data.index.values
dates_all_t = dates_all[1:]
sec1_levels = np.array(price_data[symbols[0]].values.flatten())
log_sec1 = np.log(sec1_levels)
sec1_returns = log_sec1[1:] - log_sec1[:-1]

sec2_levels = np.array(price_data[symbols[1]].values.flatten())

In [11]:
df = pd.DataFrame({'date':dates_all,'sec_level':sec2_levels})

For the grid, we choose button-export to be able to export the entire grid at will, as soon as the data changes because of filtering.

In [12]:
columnDefs = [{'field':'date', 'headerName':'Date'},
             {'field':'sec_level', 'headerName':'Value'}]

gridOptions={
    'columnDefs': columnDefs,
    'animateRows':'true',
};

process_data_grid = Grid(width=600,
             quick_filter=True,
             export_mode="buttons",
             show_toggle_edit=True,
             theme='ag-theme-balham',
             menu={'input_div_css':{'flex-direction':'column', 'flex-wrap':'nowrap'}},
             grid_options=gridOptions,
             grid_data=df)

In [13]:
sc_x = DateScale()
sc_y = LinearScale()

scatt = Scatter(x=dates_all, y=sec2_levels, scales={'x': sc_x, 'y': sc_y})
ax_x = Axis(scale=sc_x, label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', tick_format='0.0f', label='Security 2')



Here is the output of the figure :

In [14]:
Figure(marks=[scatt], axes=[ax_x, ax_y])

Figure(axes=[Axis(label='Date', scale=DateScale(), side='bottom'), Axis(label='Security 2', orientation='verti…

In [15]:
process_data_grid

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='buttons', height='350px', menu={'buttons': [{…

Now let's observe the `grid_data_out` and update the data in the figure as soon as we get the data out. You can try to filter on any value, especially on the second value to eliminate the records that are lower than a certain value for example.

In [16]:
def on_change_data_out(change):
    x = [val for val in np.array( change.new['grid']['date'])]
    y = [val for val in np.array( change.new['grid']['sec_level'])]
    x = np.array(x, dtype='datetime64[ns]')
    scatt.x = x
    scatt.y = y
    
process_data_grid.observe(on_change_data_out, names='grid_data_out')